# Load Packages


In [6]:
#using Dates                #Julia 0.7, needed for printmat.jl
include("printmat.jl")

printmatDate

# Some Important Types

Julia has many different types of variables: signed integers (like 2 or -5), floating point numbers (2.0 and -5.1), bools (false/true), bitarrays (similar to bools, but with more efficient use of memory), strings ("hello"), Dates (2017-04-23) and many more types. 

The numerical types also comes with subtypes for different precisions, for instance, Float16, Float32 and Float64. Unless you specify otherwise, code like
```
a = 2
b = 2.0
```
gives an Int64 and a Float64 respectively (at least on Windows 7, 64 bit).

## Integers and Floats

In [7]:
a = 2                   #integer, Int (Int64 on most machines)
b = 2.0                 #floating point, (Float64 on most machines)
A = [1;2]
B = [1.0;2.0]

println("a: ",typeof(a))
printmat(a)

println("b: ",typeof(b))
printmat(b)

println("A: ",typeof(A))
printmat(A)

println("B: ",typeof(B))
printmat(B)

a: Int64
         2

b: Float64
     2.000

A: Array{Int64,1}
         1
         2

B: Array{Float64,1}
     1.000
     2.000



# Why Use Int When There Are Floats? 

That is, why bother with sometimes using 3 when you could use 3.0 everywhere? Mostly because you cannot use 3.0 everywhere... For instance, you cannot pick out element x[3.0] from a vector. It has to be x[3].

In [8]:
x = [1;10;100;1000]
try 
   y = x[3.0]   
catch    
  println("no, x[3.0] does not work")    
end

no, x[3.0] does not work


## Bools and BitArrays

In [9]:
c = 2 > 1.1
println("c: ",typeof(c))
printmat(c)

C = A .> 1.5
println("\nC: ",typeof(C))
printmat(C)

println("A BitArray is a more economical array version of Bool. Notice that typeof(C[1]) gives: ",typeof(C[1]))

c: Bool
      true


C: BitArray{1}
     false
      true

A BitArray is a more economical array version of Bool. Notice that typeof(C[1]) gives: Bool


## Char and Strings

In [10]:
t = 'a'                                    #Char, just one letter
println(typeof(t))

txt = "Dogs are nicer than cats."          #String, could be a long novel 
println(typeof(txt))

Char
String


# Calulations with Mixed Types and Converting Types

A calculation like "integer" + "float" works and the type of the result will be a float (the more flexible type). Similarly, "bool" + "integer" will give an integer. These promotion rules make it easy to have mixed types in calculations, and also provide a simple way of converting a variable from one type to another. (There are also an explicit convert() function that might be quicker.)

## Some Calculations with Mixed Types ("promotion")

In [11]:
println("integer + float: ",1+2.0)                #integer + float
println("bool + integer: ",(1.>0) + 2)            #bool + integer

integer + float: 3.0
bool + integer: 3


## Converting from Int to Float and vice versa

In [12]:
x = [1.1;10.1;100.1]
println("x: ",typeof(x))
printmat(x)

B_to_int = round.(Int,x)                     #float -> integer by rounding
println("rounding x to int: ",typeof(B_to_int))
printmat(B_to_int)

println("A: ",typeof(A))
printmat(A)

A_to_float = A .+ 0.0                       #int -> float by adding 0.0
println("after converting A to float: ",typeof(A_to_float))   
printmat(A_to_float)                        #convert(Array{Float64},A) also works

x: Array{Float64,1}
     1.100
    10.100
   100.100

rounding x to int: Array{Int64,1}
         1
        10
       100

A: Array{Int64,1}
         1
         2

after converting A to float: Array{Float64,1}
     1.000
     2.000



## Converting from Bools and BitArrays to Int and vice versa

In [13]:
C_to_int = C .+ 0                        #BitArray -> int by adding 0
println(typeof(C_to_int))                #convert.(Int,C) also works
printmat(C_to_int)

D = [1;0;1]                              
D_to_bit = D .> 0                        #int -> BitArray by comparing
println(typeof(D_to_bit))                #convert(BitArray,D) also works 
printmat(D_to_bit)

Array{Int64,1}
         0
         1

BitArray{1}
      true
     false
      true



## From Bools and BitArrays to Int: A Tricky Case

```false``` is a "strong zero" in the sense that 
```false*NaN == 0``` and ```false*Inf == 0```

If you do not want that behaviour in your code, transform ```false``` to 0 and then multiply.

In [5]:
println(false*NaN)
println(false*Inf)

println(convert(Int,false)*NaN)
println(convert(Int,false)*Inf)

0.0
0.0
NaN
NaN


# Type Instability

Your code will run faster if your variables do not change type in the computations. The next cells illustrate that.

In [14]:
function fn1(n)
    x = 0                    #x starts out as an Int
    for i = 1:n
        x = x + 0.1          #x is changed from Int to a Float when you add 0.1
    end
    return x
end    

function fn2(n)
    x = 0.0                  #x starts out as a Float
    for i = 1:n
        x = x + 0.1          #x remains a float
    end
    return x
end

fn2 (generic function with 1 method)

In [15]:
x = fn1(10)                #a "dry" run makes the subsequent timing results more accurate
x = fn2(10)

@time fn1(1e+6)
@time fn2(1e+6)

println("\nfn2() is much faster and uses much less memory (even a more serious timing would show that...)")

  0.029701 seconds (2.00 M allocations: 30.592 MiB, 12.85% gc time)
  0.005716 seconds (1.33 k allocations: 66.544 KiB)

fn2() is much faster and uses much less memory (even a more serious timing would show that...)


In [16]:
@code_warntype fn1(1e+6)   #to spot type instability, run this. Notice the red warnings.

#@code_warntype fn2(1e+6)  #compare with this

Variables:
  #self# <optimized out>
  n::Float64
  i <optimized out>
  #temp#@_4::Int64
  x::Union{Float64, Int64}
  u@_6::Int64
  shift_hi <optimized out>
  shift_lo <optimized out>
  x_hi <optimized out>
  x_lo <optimized out>
  #temp#@_11 <optimized out>
  #temp#@_12 <optimized out>
  w::Float64
  u@_14::Float64
  v::Float64
  #temp#@_16::Core.MethodInstance
  #temp#@_17::Float64

Body:
  begin 
      x::Union{Float64, Int64} = 0 # line 3:
      SSAValue(0) = $(Expr(:invoke, MethodInstance for colon(::Float64, ::Float64, ::Float64), :(Base.colon), :((Base.sitofp)(Float64, 1)::Float64), :((Base.sitofp)(Float64, 1)::Float64), :(n)))
      #temp#@_4::Int64 = 1
      5: 
      unless (Base.not_int)((Base.slt_int)((Core.getfield)(SSAValue(0), :len)::Int64, #temp#@_4::Int64)::Bool)::Bool goto 52
      $(Expr(:inbounds, false))
      # meta: location twiceprecision.jl next 187
      # meta: location twiceprecision.jl unsafe_getindex 194
      u@_6::Int64 = (Base.sub_int)(#temp#@_4::Int64, 